# Model of work distribution in a flat organization

* no managers; only workers
* no support staff; all infrastructure for communication and tasks works without failure
* no dark patterns

TODO:
* use realistic distributions (e.g., power law) instead of uniform
* instead of searching randomly, search second-order contacts

In [1]:
import random

### global variables for simulation

In [2]:
skill_set_for_people = ['A','B','C'] # uniform distribution
skill_set_for_tasks  = ['A','B','C'] # uniform distribution
max_skill_level_per_person      = 3  # uniform distribution
max_skill_level_per_task        = 3  # uniform distribution
max_number_of_tasks_per_process = 3  # uniform distribution
max_task_duration_in_ticks      = 5  # uniform distribution

number_of_people = 6
number_of_processes = 5
social_circle_size = 3

max_ticks_to_simulate = 100

### The two classes: person and process

In [3]:
class person():
    # static variables
#    skill_set = ['A','B','C']
#    max_skill_level = 3
    
    def __init__(self, unique_id:int, skill_specialization_dict):
        """
        each person has a
        * uniquie_id
        * skill/specialization matrix
        """
        self.unique_id = unique_id # problem: this can be set by the user after initialization; see https://stackoverflow.com/a/48709694/1164295
        self.backlog_of_tasks = []
        self.status = "idle"
        self.assigned_task = None 
        self.contact_list = None
        self.skill_specialization_dict = {}
        for skill in skill_set_for_people:
            self.skill_specialization_dict[skill] = random.randint(1,max_skill_level_per_person+1)
        return
        
    def set_status(self, status):
        """
        status is a string with value idle xor working xor coordinating
        
        As this class is currently written, user can set "status" to arbitrary values
        """
        try:
            assert(status=="idle" or 
                   status=="working" or 
                   status=="coordinating")
            self.status = status
            return "status set to '"+status+"'"
        except AssertionError:
            return "ERROR: failed to set status; use either 'idle' xor 'working' xor 'coordinating'"
        return "ERROR: failed"
        

    def add_person_to_contact_list(self, person_id):
        """
        https://en.wikipedia.org/wiki/Dunbar%27s_number
        """
        self.contact_list.append(person_id)
        if len(self.contact_list)>social_circle_size:
            self.contact_list.pop(0)
        return
    
    #def add_task_to_backlog(self, work_item):
    #    self.backlog_of_tasks.append(work_item)
    #    return 
        
    #def get_next_task_from_backlog(self):
    #    return self.backlog_of_tasks.pop(0)
    
    #def show_skill_specialization(self):
    #    return self.skill_specialization_dict
        
    #def show_task_backlog(self):
    #    return self.backlog_of_tasks

In [4]:
new_person = person(1, {'A': 2, 'B': 0})
new_person.set_status("asdf")

"ERROR: failed to set status; use either 'idle' xor 'working' xor 'coordinating'"

In [5]:
new_person.set_status("working")

"status set to 'working'"

In [6]:
list_of_people = []
for person_index in range(number_of_people):
    this_person_dict = {}
    for specialization in skill_set_for_people:
        this_person_dict[specialization] = random.choice(range(max_skill_level_per_person))
    
    list_of_people.append(person(person_index, this_person_dict))

In [7]:
for person in list_of_people:
    print(person.unique_id)

0
1
2
3
4
5


In [8]:
class process():
    """
    processes are a sequence of tasks
    """
    # static variables
    #skill_set = ['A','B','C']
    #max_skill_level = 3
    
    
    def __init__(self, unique_id:int):
        self.unique_id = unique_id
        self.list_of_tasks = []
        # a process is a sequence of tasks
#        for task_id in range(process.max_number_of_tasks):
        for task_id in range(random.randint(1,max_number_of_tasks_per_process+1)):
            task_dict = {} #{"task id": task_id}
            # task is either completed, active, or waiting
            #task_dict['status'] = "waiting"
            # each task has a skill, specicalization, duration
            duration = random.randint(1,max_task_duration_in_ticks+1)
            task_dict = {'process ID': unique_id,
                         'specialization': random.choice(skill_set_for_tasks),
                         'skill level': random.randint(1,max_skill_level_per_task+1),
                         'total duration': duration,
                         'remaining duration': duration}
            self.list_of_tasks.append(task_dict)
   
    def show_tasks(self):
        for task_dict in self.list_of_tasks:
            print(task_dict)
        return
    
    #def show_process(self):
    #    return self.list_of_tasks
    
    #def start_next_task(self):
    #    """
    #    removes task from process
    #    """
    #    return self.list_of_tasks.pop()
    
    #def what_is_next_task(self):
    #    return self.list_of_tasks[-1]

In [9]:
new_p = process(3)
new_p.list_of_tasks

[{'process ID': 3,
  'specialization': 'A',
  'skill level': 1,
  'total duration': 1,
  'remaining duration': 1},
 {'process ID': 3,
  'specialization': 'C',
  'skill level': 3,
  'total duration': 2,
  'remaining duration': 2},
 {'process ID': 3,
  'specialization': 'A',
  'skill level': 4,
  'total duration': 5,
  'remaining duration': 5}]

In [10]:
list_of_processes = []
for process_id in range(number_of_processes):
    list_of_processes.append(process(process_id))

In [11]:
for process in list_of_processes:
    print(process.unique_id)

0
1
2
3
4


### Assessment tools

In [12]:
def show_all_people(list_of_people):
    for person in list_of_people:
        print('id=',person.unique_id, 
              '; skill matrix=',person.skill_specialization_dict)

In [13]:
show_all_people(list_of_people)

id= 0 ; skill matrix= {'A': 1, 'B': 4, 'C': 2}
id= 1 ; skill matrix= {'A': 2, 'B': 3, 'C': 4}
id= 2 ; skill matrix= {'A': 4, 'B': 4, 'C': 3}
id= 3 ; skill matrix= {'A': 2, 'B': 2, 'C': 2}
id= 4 ; skill matrix= {'A': 4, 'B': 1, 'C': 1}
id= 5 ; skill matrix= {'A': 2, 'B': 4, 'C': 3}


In [14]:
def current_status_of_people(list_of_people):
    for person in list_of_people:
        print('person id=',person.unique_id, 
                  '; status=',person.status,
                  '; number of tasks in backlog=',len(person.backlog_of_tasks),
                  '; task=',person.assigned_task)

In [15]:
current_status_of_people(list_of_people)

person id= 0 ; status= idle ; number of tasks in backlog= 0 ; task= None
person id= 1 ; status= idle ; number of tasks in backlog= 0 ; task= None
person id= 2 ; status= idle ; number of tasks in backlog= 0 ; task= None
person id= 3 ; status= idle ; number of tasks in backlog= 0 ; task= None
person id= 4 ; status= idle ; number of tasks in backlog= 0 ; task= None
person id= 5 ; status= idle ; number of tasks in backlog= 0 ; task= None


In [16]:
def cumulative_task_backlog_size(list_of_people):
    backlog_count = 0
    for person in list_of_people:
        backlog_count += len(person.backlog_of_tasks)
    return backlog_count

In [17]:
cumulative_task_backlog_size(list_of_people)

0

In [18]:
def all_idle(list_of_people):
    for person in list_of_people:
        if person.status != "idle":
            return False
    return True

In [19]:
all_idle(list_of_people)

True

In [20]:
def show_all_unassigned_tasks(list_of_processes):
    for process in list_of_processes:
        for task in process.list_of_tasks:
            print(task)
    return

In [21]:
show_all_unassigned_tasks(list_of_processes)

{'process ID': 0, 'specialization': 'A', 'skill level': 2, 'total duration': 3, 'remaining duration': 3}
{'process ID': 0, 'specialization': 'C', 'skill level': 4, 'total duration': 2, 'remaining duration': 2}
{'process ID': 0, 'specialization': 'A', 'skill level': 2, 'total duration': 4, 'remaining duration': 4}
{'process ID': 1, 'specialization': 'C', 'skill level': 3, 'total duration': 6, 'remaining duration': 6}
{'process ID': 1, 'specialization': 'C', 'skill level': 3, 'total duration': 1, 'remaining duration': 1}
{'process ID': 1, 'specialization': 'C', 'skill level': 4, 'total duration': 4, 'remaining duration': 4}
{'process ID': 1, 'specialization': 'C', 'skill level': 3, 'total duration': 4, 'remaining duration': 4}
{'process ID': 2, 'specialization': 'B', 'skill level': 2, 'total duration': 6, 'remaining duration': 6}
{'process ID': 3, 'specialization': 'A', 'skill level': 2, 'total duration': 5, 'remaining duration': 5}
{'process ID': 4, 'specialization': 'C', 'skill level':

In [22]:
def cumulative_unassigned_task_time_remaining(list_of_processes):
    total_time = 0
    for process in list_of_processes:
        for task in process.list_of_tasks:
            total_time += task['remaining duration']
    return total_time

In [23]:
print(cumulative_unassigned_task_time_remaining(list_of_processes))

42


In [24]:
def number_of_unassigned_tasks(list_of_processes):
    task_count = 0
    for process in list_of_processes:
        task_count += len(process.list_of_tasks)
    return task_count

In [25]:
number_of_unassigned_tasks(list_of_processes)

11

### Initialize Simulation

In [26]:
# randomly distribute processes to people

for index in range(len(list_of_people)):
    #print('person ID',list_of_people[index].unique_id)
    try:
        list_of_people[index].backlog_of_tasks.append(list_of_processes.pop())
    except IndexError:
        print("there are more people than processes")
        break

there are more people than processes


In [27]:
current_status_of_people(list_of_people)

person id= 0 ; status= idle ; number of tasks in backlog= 1 ; task= None
person id= 1 ; status= idle ; number of tasks in backlog= 1 ; task= None
person id= 2 ; status= idle ; number of tasks in backlog= 1 ; task= None
person id= 3 ; status= idle ; number of tasks in backlog= 1 ; task= None
person id= 4 ; status= idle ; number of tasks in backlog= 1 ; task= None
person id= 5 ; status= idle ; number of tasks in backlog= 0 ; task= None


### Begin ticks of Simulation

In [28]:
def pick_a_random_person(person_index, contacts, list_of_people):
    """
    find someone who is not myself and is not someone I already know
    """
    attempts = 0
    try:
        len(contacts)
    except TypeError: # contacts is None
        contacts=[]
    while (attempts<100):
        another_person = random.choice(range(len(list_of_people)))
        if ((another_person not in contacts) and 
            (another_person != person_index)):
            return another_person
    print("failed to find another person who is not a contact")
    return None
    

In [29]:
tick=-1

while ((number_of_unassigned_tasks(list_of_processes) > 0) or
       (cumulative_task_backlog_size(list_of_people)  > 0) or
       all_idle(list_of_people)):
    tick=tick+1
    print('\n===== tick',tick,'=====')
    show_all_unassigned_tasks(list_of_processes)
    print('number of unworked tasks:', number_of_unassigned_tasks(list_of_processes))
    print('cumulative backlog size: ', cumulative_task_backlog_size(list_of_people))
    print('all idle?:               ', all_idle(list_of_people))
    current_status_of_people(list_of_people)
    if (tick>max_ticks_to_simulate):
        print("breaking due to excessive tick count")
        break
    # each person looks in their backlog for work
    for person_index in range(len(list_of_people)):
        skill_dict = list_of_people[person_index].skill_specialization_dict
        if list_of_people[person_index].status=="idle":
            try:
                process = list_of_people[person_index].backlog_of_tasks.pop(0)
                print(person_index,'is assigned',process.list_of_tasks[0],
                      'and has skills',skill_dict)
            except IndexError: # pop from empty list
                print("person index",person_index,"has no task in backlog")
                process = None
        
        if process:
            if (process.list_of_tasks[0]['skill level'] <= skill_dict[process.list_of_tasks[0]['specialization']]): # person can do task
                speedup = skill_dict[process.list_of_tasks[0]['specialization']]/process.list_of_tasks[0]['skill level']
                print("   speedup for person",person_index,"is", speedup)
                list_of_people[person_index].status="working"
                process.list_of_tasks[0]['remaining duration'] = process.list_of_tasks[0]['remaining duration'] - speedup # "doing the work"
                if (process.list_of_tasks[0]['remaining duration']<0): # task was completed
                    list_of_people[person_index].assigned_task = None # it disappears from the simulation
                    list_of_people[person_index].status = "idle"
                else:
                    list_of_people[person_index].assigned_task = process
                
            else: # person doesn't have sufficient skill for task; person needs to find someone else
                list_of_people[person_index].status="coordinating"
                contacts = list_of_people[person_index].contact_list
                print("   contacts=",contacts)
                if contacts: # I know people!
                    for contact_id in contacts: # do the people I know have the skills to do this task?
                        contacts_skill_dict = list_of_people[contact_id].skill_specialization_dict
                        if (process.list_of_tasks[0]['skill level'] <= contacts_skill_dict[process.list_of_tasks[0]['specialization']]): # contact can do task
                            list_of_people[contact_id].backlog_of_tasks.append(process)
                            list_of_people[person_index].assigned_task = None
                            list_of_people[person_index].status = "idle"
                if list_of_people[person_index].status=="coordinating": # didn't have a contact who could do the work
                    another_person_id = pick_a_random_person(person_index, contacts, list_of_people)
                    another_person_skill_dict = list_of_people[another_person_id].skill_specialization_dict
                    if (process.list_of_tasks[0]['skill level'] <= another_person_skill_dict[process.list_of_tasks[0]['specialization']]): # another_person can do task
                        list_of_people[another_person_id].backlog_of_tasks.append(process)
                        list_of_people[person_index].assigned_task = None
                        list_of_people[person_index].status = "idle"
        


===== tick 0 =====
number of unworked tasks: 0
cumulative backlog size:  5
all idle?:                True
person id= 0 ; status= idle ; number of tasks in backlog= 1 ; task= None
person id= 1 ; status= idle ; number of tasks in backlog= 1 ; task= None
person id= 2 ; status= idle ; number of tasks in backlog= 1 ; task= None
person id= 3 ; status= idle ; number of tasks in backlog= 1 ; task= None
person id= 4 ; status= idle ; number of tasks in backlog= 1 ; task= None
person id= 5 ; status= idle ; number of tasks in backlog= 0 ; task= None
0 is assigned {'process ID': 4, 'specialization': 'C', 'skill level': 2, 'total duration': 6, 'remaining duration': 6} and has skills {'A': 1, 'B': 4, 'C': 2}
   speedup for person 0 is 1.0
1 is assigned {'process ID': 3, 'specialization': 'A', 'skill level': 2, 'total duration': 5, 'remaining duration': 5} and has skills {'A': 2, 'B': 3, 'C': 4}
   speedup for person 1 is 1.0
2 is assigned {'process ID': 2, 'specialization': 'B', 'skill level': 2, 't